# Exploratory Analisys

## Import libraries

In [6]:
import pandas as pd
import numpy as np

## Import Databases

book_rating.csv from Kaggle

In [7]:
book_rating = pd.read_csv("../data/books_rating.csv") 
print(book_rating.head())

           Id                           Title  Price         User_id  \
0  1882931173  Its Only Art If Its Well Hung!    NaN   AVCGYZL8FQQTD   
1  0826414346        Dr. Seuss: American Icon    NaN  A30TK6U7DNS82R   
2  0826414346        Dr. Seuss: American Icon    NaN  A3UH4UZ4RSVO82   
3  0826414346        Dr. Seuss: American Icon    NaN  A2MVUWT453QH61   
4  0826414346        Dr. Seuss: American Icon    NaN  A22X4XUPKF66MR   

                          profileName review/helpfulness  review/score  \
0               Jim of Oz "jim-of-oz"                7/7           4.0   
1                       Kevin Killian              10/10           5.0   
2                        John Granger              10/11           5.0   
3  Roy E. Perry "amateur philosopher"                7/7           4.0   
4     D. H. Richards "ninthwavestore"                3/3           4.0   

   review/time                                   review/summary  \
0    940636800           Nice collection of Julie Strai

In [8]:
print(book_rating.columns)
print(book_rating.dtypes)
print(book_rating["review/summary"].head())
print(book_rating["review/score"].head())

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')
Id                     object
Title                  object
Price                 float64
User_id                object
profileName            object
review/helpfulness     object
review/score          float64
review/time             int64
review/summary         object
review/text            object
dtype: object
0             Nice collection of Julie Strain images
1                                  Really Enjoyed It
2    Essential for every personal and Public Library
3    Phlip Nel gives silly Seuss a serious treatment
4                             Good academic overview
Name: review/summary, dtype: object
0    4.0
1    5.0
2    5.0
3    4.0
4    4.0
Name: review/score, dtype: float64


Merge dataset by review/text and review/score

In [9]:
merged_df = book_rating[["review/summary", "review/score"]].dropna()
merged_df.head()

,review/summary,review/score
0,Nice collection of Julie Strain images,4.0
1,Really Enjoyed It,5.0
2,Essential for every personal and Public Library,5.0
3,Phlip Nel gives silly Seuss a serious treatment,4.0
4,Good academic overview,4.0


In [10]:
print(merged_df.nunique())

review/summary    1592314
review/score            5
dtype: int64


In [9]:
merged_df.to_csv("../data/merged_reviews.csv", index=False)

### Cleaning text tools

In [11]:
import re
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer

### Option 1: Delete comments with score 3

In [15]:
merged_df_binary = merged_df[merged_df["review/score"] != 3].copy()
merged_df_binary.loc[:, "label"] = merged_df_binary["review/score"].apply(lambda x:1 if x > 3 else 0)
merged_df_binary.count()

review/summary    2745328
review/score      2745328
label             2745328
dtype: int64

In [16]:
print(merged_df_binary.head())

                                    review/summary  review/score  label
0           Nice collection of Julie Strain images           4.0      1
1                                Really Enjoyed It           5.0      1
2  Essential for every personal and Public Library           5.0      1
3  Phlip Nel gives silly Seuss a serious treatment           4.0      1
4                           Good academic overview           4.0      1


In [17]:
print(merged_df_binary.nunique())

review/summary    1453884
review/score            4
label                   2
dtype: int64


In [18]:
def count_labels(df, column_name):
    counts = df[column_name].value_counts()
    return counts.get(1, 0), counts.get(0, 0)

# Usar la función para contar los valores en la columna "review/summary"
count_ones, count_zeros = count_labels(merged_df_binary, "label")

print(f"Cantidad de 1: {count_ones}")
print(f"Cantidad de 0: {count_zeros}")

Cantidad de 1: 2392645
Cantidad de 0: 352683


In [19]:
print(merged_df_binary['review/summary'].value_counts())

review/summary
Great Book                                 6820
Great book                                 6695
Excellent                                  6492
Great                                      3886
Classic                                    3690
                                           ... 
You've got to hand it to the author...        1
Don't try to fool us with fake reviews.       1
Spectacular !                                 1
The truth about WOTWS                         1
How could someone not love this ??            1
Name: count, Length: 1453884, dtype: int64


Existe un desfasaje grande entre reseñas con puntaje 1 y 0

In [10]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [11]:
# Equilibrar los datos (Undersampling y Oversampling)
undersampler = RandomUnderSampler(sampling_strategy=1.0, random_state=42)
oversampler = SMOTE(sampling_strategy=0.5, random_state=42)

In [ ]:
import pandas as pd
import re
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Descargar stopwords de NLTK
#nltk.download('stopwords')

# Cargar datos
book_rating = pd.read_csv("../data/books_rating.csv")

# Filtrar datos solo con las columnas necesarias
df = book_rating[['review/score', 'review/text']].dropna()

# Opción 1: Eliminamos los comentarios con puntuación de 3
df_binary = df[df['review/score'] != 3].copy()
df_binary.loc[:, 'label'] = df_binary['review/score'].apply(lambda x: 1 if x > 3 else 0)

# Opción 3: Clasificación en 3 categorías
df_multi = df.copy()
df_multi.loc[:, 'label'] = df_multi['review/score'].apply(lambda x: 2 if x > 3 else (1 if x == 3 else 0))

# Preprocesamiento de texto
def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower().split()
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text if word not in set(stopwords.words('english'))]
    return ' '.join(text)

df_binary.loc[:, 'clean_text'] = df_binary['review/text'].apply(clean_text)
df_multi.loc[:, 'clean_text'] = df_multi['review/text'].apply(clean_text)

# Vectorización de texto
vectorizer = TfidfVectorizer(max_features=5000)
X_binary = vectorizer.fit_transform(df_binary['clean_text']).toarray()
y_binary = df_binary['label'].values
X_multi = vectorizer.fit_transform(df_multi['clean_text']).toarray()
y_multi = df_multi['label'].values

# Equilibrar los datos (Undersampling y Oversampling)
undersampler = RandomUnderSampler(sampling_strategy=1.0, random_state=42)
oversampler = SMOTE(sampling_strategy=0.5, random_state=42)

# Aplicar undersampling (Reducir clase mayoritaria)
X_bin_under, y_bin_under = undersampler.fit_resample(X_binary, y_binary)

# Aplicar oversampling (Aumentar clase minoritaria)
X_bin_over, y_bin_over = oversampler.fit_resample(X_binary, y_binary)

# División en entrenamiento y prueba
X_train_bin_under, X_test_bin_under, y_train_bin_under, y_test_bin_under = train_test_split(X_bin_under, y_bin_under, test_size=0.2, random_state=42)
X_train_bin_over, X_test_bin_over, y_train_bin_over, y_test_bin_over = train_test_split(X_bin_over, y_bin_over, test_size=0.2, random_state=42)
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X_multi, y_multi, test_size=0.2, random_state=42)

# Entrenar modelo Naive Bayes para ambas opciones
classifier_bin_under = MultinomialNB()
classifier_bin_under.fit(X_train_bin_under, y_train_bin_under)
y_pred_bin_under = classifier_bin_under.predict(X_test_bin_under)

classifier_bin_over = MultinomialNB()
classifier_bin_over.fit(X_train_bin_over, y_train_bin_over)
y_pred_bin_over = classifier_bin_over.predict(X_test_bin_over)

classifier_multi = MultinomialNB()
classifier_multi.fit(X_train_multi, y_train_multi)
y_pred_multi = classifier_multi.predict(X_test_multi)

# Evaluación
def evaluate_model(y_test, y_pred, title):
    print(f"{title} Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(title)
    plt.show()

evaluate_model(y_test_bin_under, y_pred_bin_under, "Binary Classification (Undersampling)")
evaluate_model(y_test_bin_over, y_pred_bin_over, "Binary Classification (Oversampling)")
evaluate_model(y_test_multi, y_pred_multi, "Multiclass Classification")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mirty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: 

In [9]:

corpus = []
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

for i in range(0, len(merged_df_binary)):
    review = re.sub('[^a-zA-Z]', ' ', merged_df_binary.iloc[i]["review/summary"])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stop_words]
    review = ' '.join(review)
    corpus.append(review)

### Create Bag of Words

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000)
X = cv.fit_transform(corpus)
y = merged_df_binary.iloc[:, 1].values

### Convert y to binary

In [29]:
y = (y > 3).astype(int)

In [47]:
print(np.unique(y))
print(np.unique(X))


[0 1]
[<Compressed Sparse Row sparse matrix of dtype 'int64'
 	with 7997961 stored elements and shape (2745328, 10000)>]


### Divide dataset into a trainning and test set

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Setting the classifier with trainning dataset

In [35]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

### Predicting result with dataset

In [36]:
y_pred = classifier.predict(X_test)

### Confusion Matrix

In [39]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 16683   4036   1940  17836]
 [  5202   6677   3410  14820]
 [  2703   2761  23975  87721]
 [  5649   2543  20220 332890]]


In [43]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6924941628146707


In [46]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calcular precisión, recall y F1-score por clase
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

print("Precisión por clase:", precision)
print("Recall por clase:", recall)
print("F1-Score por clase:", f1)

Precisión por clase: [0.55174124 0.41686958 0.48390352 0.73442364]
Recall por clase: [0.41197679 0.22176094 0.20463469 0.92136218]
F1-Score por clase: [0.47172425 0.28951134 0.28763384 0.81734021]
